In [1]:
script_content = """ 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# 🔹 تحميل بيانات الأزهار
dataset_name = "tf_flowers"
(train_data, test_data), dataset_info = tfds.load(
    dataset_name, split=["train[:80%]", "train[80%:]"], as_supervised=True, with_info=True
)

# 🔹 تجهيز البيانات
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

train_data = train_data.map(preprocess_image).batch(32).shuffle(1000)
test_data = test_data.map(preprocess_image).batch(32)

# 🔹 بناء النموذج باستخدام MobileNet
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5", trainable=False),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(dataset_info.features["label"].num_classes, activation="softmax")
])

# 🔹 تجميع النموذج
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# 🔹 إضافة سجل تدريبي
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# 🔹 تدريب النموذج
model.fit(train_data, epochs=15, validation_data=test_data, callbacks=[early_stop])

# 🔹 حفظ النموذج في المسار الجديد
model_path = "/mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/models/trained_model.h5"
model.save(model_path)

print(f"✅ تم حفظ النموذج المدرب في: {model_path}")
"""

# 🔹 حفظ السكربت داخل WSL باستخدام Jupyter Notebook
script_path = "/mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/scripts/train_model.py"

with open(script_path, "w", encoding="utf-8") as f:
    f.write(script_content)

print(f"✅ تم حفظ السكربت في: {script_path}")

✅ تم حفظ السكربت في: /mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/scripts/train_model.py


In [2]:
!python3 /mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/scripts/train_model.py

2025-05-31 18:07:25.726793: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-31 18:07:25.933430: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-31 18:07:25.933483: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-31 18:07:26.039378: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-31 18:07:26.155668: I tensorflow/core/platform/cpu_feature_gua

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# 🔹 تحميل بيانات الأزهار
dataset_name = "tf_flowers"
(train_data, test_data), dataset_info = tfds.load(
    dataset_name, split=["train[:80%]", "train[80%:]"], as_supervised=True, with_info=True
)

# 🔹 تجهيز البيانات
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

train_data = train_data.map(preprocess_image).batch(32).shuffle(1000)
test_data = test_data.map(preprocess_image).batch(32)

# 🔹 استخدام Functional API مع Lambda Layer
input_layer = tf.keras.layers.Input(shape=(224, 224, 3))
hub_layer = hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5", trainable=False)
processed_hub = tf.keras.layers.Lambda(lambda x: hub_layer(x))(input_layer)  # ✅ الحل هنا
flatten_layer = tf.keras.layers.Flatten()(processed_hub)
dense_layer = tf.keras.layers.Dense(128, activation="relu")(flatten_layer)
dropout_layer = tf.keras.layers.Dropout(0.3)(dense_layer)
output_layer = tf.keras.layers.Dense(dataset_info.features["label"].num_classes, activation="softmax")(dropout_layer)

# 🔹 إنشاء النموذج
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# 🔹 تجميع النموذج
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# 🔹 تدريب النموذج
model.fit(train_data, epochs=15, validation_data=test_data)

# 🔹 حفظ النموذج
model_path = "/mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/models/trained_model.h5"
model.save(model_path)

print(f"✅ تم حفظ النموذج المدرب في: {model_path}")

Epoch 1/15


2025-05-31 18:12:13.551303: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 51 of 1000
2025-05-31 18:12:26.073250: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
I0000 00:00:1748704346.295265    2591 service.cc:145] XLA service 0x7f9ae8024030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748704346.311370    2591 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce MX450, Compute Capability 7.5
2025-05-31 18:12:27.147647: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-05-31 18:12:29.864010: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2025-05-31 18:12:39.444732: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=2} for c

92/92 ━━━━━━━━━━━━━━━━━━━━ 95s 449ms/step - accuracy: 0.6862 - loss: 0.9089 - val_accuracy: 0.8978 - val_loss: 0.2999
Epoch 2/15


2025-05-31 18:13:45.983538: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 70 of 1000
2025-05-31 18:13:47.713112: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 27s 128ms/step - accuracy: 0.8999 - loss: 0.2738 - val_accuracy: 0.9019 - val_loss: 0.2594
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 51s 449ms/step - accuracy: 0.9378 - loss: 0.1813 - val_accuracy: 0.9114 - val_loss: 0.2578
Epoch 4/15


...

2025-05-31 18:15:51.310409: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 33 of 1000
2025-05-31 18:16:01.377562: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 70 of 1000
2025-05-31 18:16:06.214969: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 47s 237ms/step - accuracy: 0.9720 - loss: 0.0910 - val_accuracy: 0.9060 - val_loss: 0.2603
Epoch 6/15


2025-05-31 18:16:38.783971: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 18 of 1000
2025-05-31 18:16:50.426674: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 35 of 1000
2025-05-31 18:17:09.907423: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:39: Filling up shuffle buffer (this may take a while): 68 of 1000
2025-05-31 18:17:19.898536: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 83s 344ms/step - accuracy: 0.9769 - loss: 0.0819 - val_accuracy: 0.9142 - val_loss: 0.2594
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 149ms/step - accuracy: 0.9861 - loss: 0.0577 - val_accuracy: 0.9060 - val_loss: 0.2953
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 116ms/step - accuracy: 0.9802 - loss: 0.0659 - val_accuracy: 0.9128 - val_loss: 0.2883
Epoch 9/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.9854 - loss: 0.0491 - val_accuracy: 0.9169 - val_loss: 0.2824
Epoch 10/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 116ms/step - accuracy: 0.9933 - loss: 0.0329 - val_accuracy: 0.9196 - val_loss: 0.2927
Epoch 11/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.9945 - loss: 0.0302 - val_accuracy: 0.9142 - val_loss: 0.3104
Epoch 12/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 121ms/step - accuracy: 0.9926 - loss: 0.0297 - val_accuracy: 0.9101 - val_loss: 0.3231
Epoch 13/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 13s 129ms/step - accuracy: 0.9951 - loss: 0.0270 - val_accuracy: 0

✅ تم حفظ النموذج المدرب في: /mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/models/trained_model.h5


In [5]:
script_content = """ 
import tensorflow as tf
import tensorflow_datasets as tfds

#   تحميل بيانات الأزهار
dataset_name = "tf_flowers"
(train_data, test_data), dataset_info = tfds.load(
    dataset_name, split=["train[:80%]", "train[80%:]"], as_supervised=True, with_info=True
)

#   تجهيز البيانات
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

train_data = train_data.map(preprocess_image).batch(32).shuffle(1000)
test_data = test_data.map(preprocess_image).batch(32)

print("✅ تم تحميل البيانات ومعالجتها بنجاح!")
"""

#  حفظ السكربت داخل `scripts/`
script_path = "/mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/scripts/data_loader.py"

with open(script_path, "w", encoding="utf-8") as f:
    f.write(script_content)

print(f"✅ تم حفظ السكربت في: {script_path}")

✅ تم حفظ السكربت في: /mnt/c/Users/ali/Desktop/STEAM_Nexus_Hub/Classification_of_flower_structure_TM1/Plant_Recognition_Lab/scripts/data_loader.py
